# Iris Classification Canary Deployment

### Import relevant packages

In [ ]:
!pip install seldon-deploy-sdk

In [ ]:
from seldon_deploy_sdk import Configuration, ApiClient, SeldonDeploymentsApi
from seldon_deploy_sdk.auth import OIDCAuthenticator

from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score, confusion_matrix

import numpy as np
import pandas as pd

import xgboost as xgb

import os
import joblib

### Loading the data

In [ ]:
dataset = load_iris()
feature_names = dataset.feature_names
class_names = list(dataset.target_names)

X = dataset.data
y = dataset.target

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Model training

#### Logistic Regressor 

In [ ]:
lr = LogisticRegression(max_iter=4000)
lr.fit(X_train, y_train)

print(precision_score(y_test, lr.predict(X_test), average="macro"))
print(recall_score(y_test, lr.predict(X_test), average="macro"))
print(accuracy_score(y_test, lr.predict(X_test)))

#### XGBoost

In [ ]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

In [ ]:
param = {
    'max_depth': 3,
    'eta': 0.3,
    'objective': 'multi:softprob',
    'num_class': 3} 

num_round = 20

In [ ]:
bst = xgb.train(param, dtrain, num_round)
preds = np.asarray([np.argmax(line) for line in bst.predict(dtest)])
print(precision_score(y_test, preds, average="macro"))
print(recall_score(y_test, preds, average="macro"))
print(accuracy_score(y_test, preds))

### Save models

In [ ]:
# Saving the logistic regressor
joblib.dump(lr, 'model.joblib')

# Saving the XGBoost classifier
bst.save_model('model.bst')

### Push model artefacts to S3

You will now need to push the saved model binaries to an s3 compatible bucket, where they can be picked up by Seldon and deployed onto Kubernetes. 

Here is an example of pushing to GCP:

In [ ]:
YOUR_NAME = ""
YOUR_BUCKET = ""

In [ ]:
!gsutil cp model.joblib gs://"{YOUR_BUCKET}"/"{YOUR_NAME}"/lr/model.joblib
!gsutil cp model.bst gs://"{YOUR_BUCKET}"/"{YOUR_NAME}"/xgb/model.bst

### Model Deployment

Deploying the model to a Seldon Deploy trial instance using the `seldon-deploy-sdk`. 

First, set up the configuration and authentication required to access the cluster. 

In [ ]:
SD_IP = "YOUR_IP"
username = "YOUR_USERNAME"
password = "YOUR_PASSWORD"

config = Configuration()
config.host = f"http://{SD_IP}/seldon-deploy/api/v1alpha1"
config.oidc_client_id = "sd-api"
config.oidc_server = f"http://{SD_IP}/auth/realms/deploy-realm"


def auth():
    auth = OIDCAuthenticator(config)
    config.access_token = auth.authenticate(username, password)
    api_client = ApiClient(config)
    return api_client

### Canary Deployment


In [ ]:
NAMESPACE = "YOUR_NAMESPACE"

MODEL_NAME = "lr"
DEPLOYMENT_NAME = "DEPLOYMENT_NAME"
MODEL_LOCATION = f"gs://{YOUR_BUCKET}/{YOUR_NAME}/lr"
PREPACKAGED_SERVER = "SKLEARN_SERVER"
DEFAULT_TRAFFIC = 60
CPU_REQUESTS = "1"
MEMORY_REQUESTS = "1Gi"
CPU_LIMITS = "1"
MEMORY_LIMITS = "1Gi"

CANARY_MODEL_NAME = 'xgb'
CANARY_DEPLOYMENT_NAME = "CANARY_NAME"
CANARY_MODEL_LOCATION = f"gs://{YOUR_BUCKET}/{YOUR_NAME}/xgb"
CANARY_PREPACKAGED_SERVER = "XGBOOST_SERVER"
CANARY_TRAFFIC = 40

In [ ]:
mldeployment = {
    "kind": "SeldonDeployment",
    "metadata": {
        "name": DEPLOYMENT_NAME,
        "namespace": NAMESPACE,
        "labels": {
            "fluentd": "true"
        }
    },
    "apiVersion": "machinelearning.seldon.io/v1alpha2",
    "spec": {
        "name": DEPLOYMENT_NAME,
        "annotations": {
            "seldon.io/engine-seldon-log-messages-externally": "true"
        },
        "protocol": "seldon",
        "transport": "rest",
        "predictors": [
            {
                "componentSpecs": [
                    {
                        "spec": {
                            "containers": [
                                {
                                    "name": f"{DEPLOYMENT_NAME}-container",
                                    "resources": {
                                        "requests": {
                                            "cpu": CPU_REQUESTS,
                                            "memory": MEMORY_REQUESTS
                                        },
                                        "limits": {
                                            "cpu": CPU_LIMITS,
                                            "memory": MEMORY_LIMITS
                                        }
                                    }
                                }
                            ]
                        }
                    }
                ],
                "name": "default",
                "replicas": 1,
                "traffic": DEFAULT_TRAFFIC,
                "graph": {
                    "implementation": PREPACKAGED_SERVER,
                    "modelUri": MODEL_LOCATION,
                    "name": f"{DEPLOYMENT_NAME}-container",
                    "endpoint": {
                        "type": "REST"
                    },
                    "parameters": [],
                    "children": [],
                    "logger": {
                        "mode": "all"
                    }
                }
            },{
                "componentSpecs": [
                    {
                        "spec": {
                            "containers": [
                                {
                                    "name": f"{CANARY_DEPLOYMENT_NAME}-container",
                                    "resources": {}
                                }
                            ]
                        }
                    }
                ],
                "name": "canary",
                "replicas": 1,
                "traffic": CANARY_TRAFFIC,
                "graph": {
                    "implementation": CANARY_PREPACKAGED_SERVER,
                    "modelUri": CANARY_MODEL_LOCATION,
                    "name": f"{CANARY_DEPLOYMENT_NAME}-container"
                }
            }
        ]
    },
    "status": {}
}

In [ ]:
deployment_api = SeldonDeploymentsApi(auth())
deployment_api.update_seldon_deployment(namespace=NAMESPACE, name=DEPLOYMENT_NAME, mldeployment=mldeployment)

Canary should now be running. You can now log into Seldon Deploy and test your deployment:

You can now test your models with this request.
```
{
    "data": {
    "names": ["Sepal length","Sepal width","Petal length", "Petal Width"],
    "ndarray": [
        [6.8,  2.8,  4.8,  1.4]
    ]
    }
}
```
  

### Get current deployments



In [ ]:
deployment_api = SeldonDeploymentsApi(auth())
deployment_api.list_seldon_deployments(namespace=NAMESPACE)

### Promote XGBoost

To promote the XGBoost classifier we can adjust our ```mldeployment``` dictionary by replacing our default scikit-learn model with the xgboost model, adjusting the traffic to 100% and removing the canary:

In [ ]:
mldeployment['spec']['predictors'][0] = mldeployment['spec']['predictors'][1]
mldeployment['spec']['predictors'][0]['traffic'] = 100
del mldeployment['spec']['predictors'][1]

In [ ]:
deployment_api = SeldonDeploymentsApi(auth())
deployment_api.create_seldon_deployment(namespace=NAMESPACE, mldeployment=mldeployment)